In [103]:
import pandas as pd
import oanda_utils
import plotly.graph_objects as go
import oanda_instrument


In [104]:
pair = "CAD_CHF"
granularity = "H1"
# ma_list = [8,16,32,64,128,256]
ma_list = [16,64] # Cross of MAs
i_pair = oanda_instrument.Instrument.get_instrument_by_name("EUR_USD")


In [105]:
df = pd.read_pickle(oanda_utils.get_his_data_filename(pair, granularity))

In [106]:
df.head()


,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2021-04-13T09:00:00.000000000Z,437,0.73453,0.73560,0.73409,0.73540,0.73440,0.73549,0.73398,0.73528,0.73466,0.73570,0.73420,0.73551
1,2021-04-13T09:00:00.000000000Z,437,0.73453,0.73560,0.73409,0.73540,0.73440,0.73549,0.73398,0.73528,0.73466,0.73570,0.73420,0.73551
2,2021-04-13T09:00:00.000000000Z,437,0.73453,0.73560,0.73409,0.73540,0.73440,0.73549,0.73398,0.73528,0.73466,0.73570,0.73420,0.73551
3,2021-04-13T10:00:00.000000000Z,386,0.73542,0.73552,0.73460,0.73460,0.73532,0.73541,0.73450,0.73450,0.73552,0.73563,0.73471,0.73471
4,2021-04-13T10:00:00.000000000Z,386,0.73542,0.73552,0.73460,0.73460,0.73532,0.73541,0.73450,0.73450,0.73552,0.73563,0.73471,0.73471


In [107]:
df.describe()


,volume
count,14997.000000
mean,693.997600
std,803.221043
min,8.000000
25%,280.000000
50%,506.000000
75%,855.000000
max,28789.000000


In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14997 entries, 0 to 14996
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   time    14997 non-null  object
 1   volume  14997 non-null  int64 
 2   mid_o   14997 non-null  object
 3   mid_h   14997 non-null  object
 4   mid_l   14997 non-null  object
 5   mid_c   14997 non-null  object
 6   bid_o   14997 non-null  object
 7   bid_h   14997 non-null  object
 8   bid_l   14997 non-null  object
 9   bid_c   14997 non-null  object
 10  ask_o   14997 non-null  object
 11  ask_h   14997 non-null  object
 12  ask_l   14997 non-null  object
 13  ask_c   14997 non-null  object
dtypes: int64(1), object(13)
memory usage: 1.6+ MB


In [109]:
non_cols = ['time', 'volume']

In [110]:
non_cols

['time', 'volume']

In [111]:
mod_cols = [x for x in df.columns if x not in non_cols]

In [112]:
mod_cols

['mid_o',
 'mid_h',
 'mid_l',
 'mid_c',
 'bid_o',
 'bid_h',
 'bid_l',
 'bid_c',
 'ask_o',
 'ask_h',
 'ask_l',
 'ask_c']

In [113]:
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [114]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [115]:
df_ma.head(10)

,time,mid_o,mid_h,mid_l,mid_c
0,2021-04-13T09:00:00.000000000Z,0.73453,0.73560,0.73409,0.73540
1,2021-04-13T09:00:00.000000000Z,0.73453,0.73560,0.73409,0.73540
2,2021-04-13T09:00:00.000000000Z,0.73453,0.73560,0.73409,0.73540
3,2021-04-13T10:00:00.000000000Z,0.73542,0.73552,0.73460,0.73460
4,2021-04-13T10:00:00.000000000Z,0.73542,0.73552,0.73460,0.73460
5,2021-04-13T10:00:00.000000000Z,0.73542,0.73552,0.73460,0.73460
6,2021-04-13T11:00:00.000000000Z,0.73457,0.73469,0.73272,0.73281
7,2021-04-13T11:00:00.000000000Z,0.73457,0.73469,0.73272,0.73281
8,2021-04-13T11:00:00.000000000Z,0.73457,0.73469,0.73272,0.73281
9,2021-04-13T12:00:00.000000000Z,0.73278,0.73342,0.73170,0.73258


In [116]:
df_ma.mid_o

0        0.73453
1        0.73453
2        0.73453
3        0.73542
4        0.73542
          ...   
14992    0.72912
14993    0.72912
14994    0.72852
14995    0.72852
14996    0.72852
Name: mid_o, Length: 14997, dtype: float64

In [117]:
df_ma['mid_o']

0        0.73453
1        0.73453
2        0.73453
3        0.73542
4        0.73542
          ...   
14992    0.72912
14993    0.72912
14994    0.72852
14995    0.72852
14996    0.72852
Name: mid_o, Length: 14997, dtype: float64

In [118]:
ma_list

[16, 64]

In [119]:
for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
    df_ma.dropna(inplace=True)
    df_ma.reset_index(drop=True, inplace=True)

In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14997 entries, 0 to 14996
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   time    14997 non-null  object 
 1   volume  14997 non-null  int64  
 2   mid_o   14997 non-null  float64
 3   mid_h   14997 non-null  float64
 4   mid_l   14997 non-null  float64
 5   mid_c   14997 non-null  float64
 6   bid_o   14997 non-null  float64
 7   bid_h   14997 non-null  float64
 8   bid_l   14997 non-null  float64
 9   bid_c   14997 non-null  float64
 10  ask_o   14997 non-null  float64
 11  ask_h   14997 non-null  float64
 12  ask_l   14997 non-null  float64
 13  ask_c   14997 non-null  float64
dtypes: float64(12), int64(1), object(1)
memory usage: 1.6+ MB


In [121]:
df.describe()

,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
count,14997.000000,14997.000000,14997.000000,14997.000000,14997.000000,14997.000000,14997.000000,14997.000000,14997.000000,14997.000000,14997.000000,14997.000000,14997.000000
mean,693.997600,0.733153,0.733640,0.732652,0.733152,0.732989,0.733491,0.732481,0.732996,0.733317,0.733810,0.732802,0.733307
std,803.221043,0.009164,0.009125,0.009206,0.009164,0.009169,0.009129,0.009208,0.009167,0.009160,0.009124,0.009204,0.009163
min,8.000000,0.709640,0.710340,0.708840,0.709700,0.709530,0.710230,0.708730,0.709580,0.709760,0.710460,0.708950,0.709810
25%,280.000000,0.726300,0.726840,0.725800,0.726290,0.726110,0.726710,0.725630,0.726130,0.726450,0.727010,0.725940,0.726450
50%,506.000000,0.733160,0.733660,0.732640,0.733140,0.733020,0.733500,0.732490,0.733010,0.733310,0.733800,0.732790,0.733300
75%,855.000000,0.741540,0.742130,0.741050,0.741500,0.741370,0.741980,0.740820,0.741380,0.741680,0.742260,0.741180,0.741650
max,28789.000000,0.751350,0.752280,0.751070,0.751360,0.751220,0.752160,0.750950,0.751240,0.751480,0.752410,0.751190,0.751480


In [122]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2021-04-14T11:00:00.000000000Z,0.73479,0.73549,0.73437,0.7346,0.733717,0.734017
1,2021-04-14T11:00:00.000000000Z,0.73479,0.73549,0.73437,0.7346,0.733791,0.734014
2,2021-04-14T11:00:00.000000000Z,0.73479,0.73549,0.73437,0.7346,0.733864,0.734011
3,2021-04-14T12:00:00.000000000Z,0.73459,0.73472,0.73326,0.7334,0.733863,0.733989
4,2021-04-14T12:00:00.000000000Z,0.73459,0.73472,0.73326,0.7334,0.733901,0.733970


MA Cross 16 & 64


In [123]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return 1
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return -1
    return 0

In [124]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)

df_trades = df_ma[df_ma.IS_TRADE!=0].copy()
df_trades.shape


(306, 10)

In [125]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
6,2021-04-14T13:00:00.000000000Z,0.73342,0.73500,0.73342,0.73496,0.734073,0.733957,0.000116,-0.000013,1
84,2021-04-15T15:00:00.000000000Z,0.73565,0.73588,0.73450,0.73586,0.737600,0.737657,-0.000057,0.000093,-1
158,2021-04-16T15:00:00.000000000Z,0.73524,0.73599,0.73506,0.73580,0.735199,0.735178,0.000020,-0.000036,1
208,2021-04-19T08:00:00.000000000Z,0.73433,0.73456,0.73296,0.73299,0.735322,0.735477,-0.000155,0.000009,-1
267,2021-04-20T04:00:00.000000000Z,0.73171,0.73197,0.73159,0.73178,0.731347,0.731266,0.000081,-0.000055,1


In [126]:
df_trades["DELTA"] = (df_trades.mid_c.diff() / i_pair.pipLocation).shift(-1)
df_trades["GAIN"] = df_trades["DELTA"] * df_trades["IS_TRADE"]

In [132]:
df_trades["GAIN"].sum()

-345.99999999999307

Take last 100 values to the latest, but only copy don't touch actual data

In [133]:
# Alternate, first is whole DF, second is DF for MA
# df_plot = df.iloc[-100:].copy()
# df_plot = df_ma.iloc[-500:].copy()
# df_plot = df_ma.iloc[50:200].copy() # trying to get a cross in the middle
df_plot = df_ma.iloc[100:450].copy() # trying to get a cross in the middle

In [134]:
df_plot.shape

(350, 10)

In [143]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x = df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor='#CC2E3C',
    increasing_line_color='#2EC886',
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10, r=10, b=10, t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e", 
    plot_bgcolor="#1e1e1e")

fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)

fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True)

fig.show()